In [3]:
import cv2
import numpy as np

In [4]:
# Calculate skew angle of an image
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=5)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    minAreaRect = cv2.minAreaRect(largestContour)

    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle

In [5]:
# Rotate the image around its center
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage

# Deskew image
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

In [6]:
im = cv2.imread('./test_3.png')

In [ ]:
im_desk = deskew(im)
im_desk_gray = cv2.cvtColor(im_desk, cv2.COLOR_BGR2GRAY)

In [8]:
plt.figure(figsize=(10,10))
plt.imshow(im_desk_gray)

NameError: name 'im_desk_gray' is not defined

<Figure size 720x720 with 0 Axes>

In [9]:
im = cv2.imread('data/imagesprojets5/')
im_desk = deskew(im)


AttributeError: 'NoneType' object has no attribute 'copy'

In [10]:
from os import listdir
from os.path import isfile, join
mypath = 'data/imagesprojets5/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [11]:
onlyfiles

['20201030_165845.jpg',
 '20201030_165849.jpg',
 '20201030_165858.jpg',
 '20201030_170611.jpg',
 '20201030_170625.jpg',
 '20201030_170721.jpg',
 '20201030_170737.jpg']

In [12]:
all_images_desk = []
for image_file in onlyfiles:
    im = cv2.imread(f'data/imagesprojets5/{image_file}')
    im_desk = deskew(im)
    all_images_desk.append(im_desk)
#     plt.figure(figsize=(50,40))
#     plt.imshow(im_desk)
#     plt.show()
    


In [17]:

def get_new(old):
    new = np.ones(old.shape, np.uint8)
    cv2.bitwise_not(new,new)
    return new

def detect_image(orig):
    # these constants are carefully picked
    MORPH = 9
    CANNY = 84
    HOUGH = 25

    img = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    cv2.GaussianBlur(img, (3,3), 0, img)


    # this is to recognize white on white
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(MORPH,MORPH))
    dilated = cv2.dilate(img, kernel)

    edges = cv2.Canny(dilated, 0, CANNY, apertureSize=3)

    lines = cv2.HoughLinesP(edges, 1,  3.14/180, HOUGH)
    for line in lines[0]:
         cv2.line(edges, (line[0], line[1]), (line[2], line[3]),
                         (255,0,0), 2, 8)

    # finding contours
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #cv2.CV_RETR_EXTERNAL,cv2.CV_CHAIN_APPROX_TC89_KCOS)
                                   
    contours = filter(lambda cont: cv2.arcLength(cont, False) > 100, contours)
    contours = filter(lambda cont: cv2.contourArea(cont) > 10000, contours)

    # simplify contours down to polygons
    rects = []
    for cont in contours:
        rect = cv2.approxPolyDP(cont, 40, True).copy().reshape(-1, 2)
        rects.append(rect)

    # that's basically it
    cv2.drawContours(orig, rects,-1,(0,255,0),1)

    # show only contours
    new = get_new(img)
    cv2.drawContours(new, rects,-1,(0,255,0),1)
    cv2.GaussianBlur(new, (9,9), 0, new)
    new = cv2.Canny(new, 0, CANNY, apertureSize=3)
#     return orig,dilated,edges,new
    cv2.namedWindow('result', cv2.WINDOW_NORMAL)
    cv2.imshow('result', orig)
    cv2.waitKey(0)
    cv2.imshow('result', dilated)
    cv2.waitKey(0)
    cv2.imshow('result', edges)
    cv2.waitKey(0)
    cv2.imshow('result', new)
    cv2.waitKey(0)

    cv2.destroyAllWindows()

In [18]:
orig = cv2.imread(f'data/imagesprojets5/{onlyfiles[0]}')
detect_image(orig)

In [48]:
cv2.CV_RETR_EXTERNAL

AttributeError: module 'cv2' has no attribute 'CV_RETR_EXTERNAL'